# 8. Applying Machine Learning to Sentiment Analysis

이 챕터에서는 **Natural Language Processing (NLP)**의 한 분야인 **sentiment analysis**를 수행할 것이다. 수행할 주제들은 다음과 같다.

- 텍스트 데이터 클리닝
- 텍스트 문서에서 feature vector 만들기
- 긍정/부정 영화 리뷰 분류
- `out-of-core` learning을 이용한 큰 텍스트 데이터 분석

## Obtaining the IMDb movie review dataset

Sentiment analysis는 **opinion mining**이라고도 불린다. 이것은 문서의 **polarity**를 분석한다. 주로 분석하는 대상은 특정 주제에 대한 저자의 의견이나 감정 표현을 분류하는 것이다.

이 챕터에서는 **Internet Movie Database (IMDb)**를 사용해 분석을 시행할 것이다. 여기에는 50,000개의 양극으로 나뉜 리뷰가 들어있다(positive or negative). positive가 의미하는 것은 영화 별점이 6점 이상, 그리고 negative는 5개 이하를 의미한다. 

*출처: http://ai.stanford.edu/~amaas/data/sentiment/ * // 다운로드 후, 해당 디렉토리로 이동해 `tar -zxf aclImdb_v1.tar`명령어로 압축 해제 가능하다.



데이터를 불러와서, 하나의 csv파일로 만들어 준다. 이를 pandas의 `DataFrame` 오브젝트로 만들어 준다. 완료까지의 시간을 측정하고 progress를 보기 위해 **PyPrind**라는 패키지를 사용한다. 

*출처: https://pypi.python.org/pypi/PyPrind/*  // `pip install pyprind`를 이용해 다운 가능하다.

In [1]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(50000)

labels = {'pos':1, 'neg':0}

df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path ='/Users/WooJin/Documents/ML_study/PML2/PML_Chp8/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile: #??
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:56


합쳐진 데이터셋은 이미 정렬 되어있기 때문에 `np.random`의 `permutation`함수를 이용해 데이터들을 섞어줄 것이다. 이는 데이터를 로컬 자체에서 훈련 / 테스트 데이터셋으로 나눠주는데 유용하다. 일단은, 섞어진 데이터를 csv파일로도 저장한다.

In [2]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('/Users/WooJin/Documents/ML_study/PML2/PML_Chp8/movie_data.csv', index = False)

In [3]:
df = pd.read_csv('/Users/WooJin/Documents/ML_study/PML2/PML_Chp8/movie_data.csv')
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


## Introducting the bag-of-words model

ch4에서 기억하듯이, 우리는 범주형 데이터를 numerical form으로 바꿔 주어야 한다. 이 섹션에서는 **bag-of-words**라는 모델을 소개할 것이다. 이 모델은 텍스트를 numerical feature vectors로 바꿔주는 역할을 한다. 이 모델의 아이디어는 다음과 같다.

1. We create a **vocabulary** of unique **tokens** - for example, words - from the entire set of documents.
2. We construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

각 문서의 고유한 단어들은 bag-of-words 단어들의 작은 부분집합으로 나타나, feature vector는 거의 0으로 이루어질 것이다(**sparse**). 

## Transforming words into feautre vectors

각 문서의 단어 카운트에 기반해 bag-of-words 모델을 만들기 위해서 scikit-learn의 `CountVectorizer`클래스를 사용한다. `CountVectorizer`클래스는 텍스트 데이터의 어레이(문서 혹은 하나의 문장)를 받아 bag-of-words 모델을 만든다.

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])

bag = count.fit_transform(docs)

`CountVectorizer`의 `fit_transform`메소드를 통해, 다음 세개의 문장을 sparse feature vectors로 만들어 주었다.

1. `The sun is shining`
2. `The weather is sweet`
3. `The sun is shining and the weather is sweet`

In [19]:
print(count.vocabulary_) #각 integer가 feature vector array의 열번호

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


우리가 만든 feature vectors를 프린트하면

In [17]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [53]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = df['review']

bag = count.fit_transform(docs)

In [58]:
import operator

In [67]:
print(count.vocabulary_[max(count.vocabulary_, key=count.vocabulary_.get)])

104082


Feature vectors의 이러한 값들은 **raw term frequencies**: tf(t,d) - the number of times a term *t* occurs in a document *d* - 라고도 불린다.

   우리가 방금 만든 bag-of-words 모델은 **1-gram**혹은 **unigram**방식이다. 일반적으로, NLP에서 어떤 item의 연속적인 sequence는 **n-gram**이라고 불린다. 
   - **1-gram**: "the", "sun", "is", "shining"
   - **2-gram**: "the sun", "sun is", "is shining"
   
     `CountVectorizer`클래스는 `ngram_range`파라메터로 이 gram을 설정할 수 있게 해준다.       ex) ngram_range(2,2)

## Assessing word relevancy via term frequency-inverse document frequency

텍스트 데이터를 분석할때, 우리는 여러 문서(negative, positive)에서 반복적으로 발견되는 어휘들에 마주치게 된다. 그러한 어휘들은 보통 유용한 정보를 담고있지 않다. 이 섹션에서는 그러한 어휘들의 가중치를 줄여주는 **term  frequency-inverse document frequency(tf-idf)**라는 테크닉을 학습할 것이다. tf-idf는 **term frequency**와 **inverse document frequency**의 곱으로 표현된다.

<img src="ch8_1.png" alt="Drawing" style="width: 350px;"/>

여기서 tf(t,d)는 term frequency이며, idf(t,d)는 다음과 같이 계산된다.

<img src="ch8_2.png" alt="Drawing" style="width: 300px;"/>

n_d 는 문서의 총 갯수, df(d,t)는 단어 t를 담고있는 문서의 수 d 이다. 1을 분모에 더해 준것은 선택적이며, 0이 아닌 값을 assign해주기 위해 쓰였다. log는 빈도가 낮은 문서에 가중치를 너무 많이 주지 않기 위해 쓰였다. 

Scikit-learn은 `CountVectorizer`에서 raw term frequencies를 가져와 tf-idfs로 만들어 주는 `TfidfTransformer`트랜스포머도 가지고 있다.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.56  0.56  0.    0.43  0.  ]
 [ 0.    0.43  0.    0.    0.56  0.43  0.56]
 [ 0.4   0.48  0.31  0.31  0.31  0.48  0.31]]


is는 3번째 문서에서 가장 term frequency가 큰 단어였다. 하지만 tf-idfs이후 is가 3번째 문서에서 상대적으로 적은(0.31이 아니라 0.48인듯?) tf-idf를 가진 것을 볼 수 있다. 하지만 우리가 아까의 공식으로 직접 계산하는 것과 `TfidfTransformer`를 써서 계산하는 것의 결과값은 조금 달랐다. scikit-learn에서의 tf-idf와 idf공식은 다음과 같다.

<img src="ch8_3.png" alt="Drawing" style="width: 350px;"/>

<img src="ch8_4.png" alt="Drawing" style="width: 400px;"/>

보통 tf-idfs를 계산하기 전에 raw term frequencies를 normalize해주는 것 또한 일반적이지만, `TfidfTransformer`는 tf-idfs를 바로 normalizes해준다. 기본적으로(`norm='12'`), scikit-learn의 `TfidfTransformer`는 L2-normalization을 적용한다. 

<img src="ch8_5.png" alt="Drawing" style="width: 500px;"/>

`TfidfTransformer`가 어떻게 적용되는지 이해하기 위해 3번째 문서안에 있는 is의 tf-idf를 어떻게 계산했는지 예시를 이용해보자.

`is`는 3번째 문서에서 2의 term frequency를 가지며, `is`의 df(d,t)는 3이다. 따라서, 우리는 다음과 같이 idf를 계산할 수 있다.

<img src="ch8_6.png" alt="Drawing" style="width: 400px;"/>

<img src="ch8_7.png" alt="Drawing" style="width: 400px;"/>

우리가 이 계산을 3번째 문서의 모든 단어들에 적용했을때, 우리는 다음과 같은 tf-idf 벡터를 얻을 수 있다: [1.69, 2.00, 1.29, 1.29, 1.29, 2.00, and 1.29]. 이것은 L2- normalization을 통해 다음과 같이 나타난다.

<img src="ch8_8.png" alt="Drawing" style="width: 700px;"/>

좀더 큰 사이즈로 실험 해보았다.

In [1]:
import rpy2

In [2]:
%load_ext rpy2.ipython

In [25]:
%%R

docs <- data.frame(matrix(c(round(runif(200, min=0, max=10))), ncol=15, byrow=T))
docs[docs == 3 ] <- 0
docs[docs == 5 ] <- 0
docs[docs == 7 ] <- 0
docs[docs == 8 ] <- 0
docs[docs == 10 ] <- 0

docs['X1'] <- 0
docs['X5'] <- 0

docs[1,1] <- 1
docs[4,1] <- 5

docs[3,5] <- 2
docs[7,5] <- 9

tf_idf <- function(x) {
  
  # create empty matrix
  tf <- data.frame(matrix(, nrow = nrow(x) , ncol = length(x)))
  df <- data.frame(matrix(, nrow = nrow(x) , ncol = length(x)))
  idf <- data.frame(matrix(, nrow = nrow(x) , ncol = length(x)))
  tfidf <- data.frame(matrix(, nrow = nrow(x) , ncol = length(x)))
  
  n_d <- nrow(x)
  
  # compute each df, tf, idf and tfidf
  for(k in 1:nrow(x)) {
    for(i in 1:length(x)) {
      
      df[k,i] <- nrow(x) - sum(x[,i] == 0)
      tf[k,i] <- x[k,i]
      idf[k,i] <- log((n_d+1)/(1+df[k,i])) 
      
      tfidf[k,i] <- tf[k,i] * (idf[k,i] + 1)
    }
  }  
  
  # L2 normalization
  norm <- c()
  for(p in 1:nrow(x)) {
    
    norm[p] <- (sum((tfidf[p, ]^2)) )^(1/2)
    
    tfidf[p, ] <- tfidf[p, ] / norm[p]
  }
  
  return(tfidf)
}

round(tf_idf(docs), digits=2)


     X1   X2   X3   X4   X5   X6   X7   X8   X9  X10  X11  X12  X13  X14  X15
1  0.19 0.00 0.52 0.00 0.00 0.22 0.00 0.00 0.28 0.00 0.00 0.73 0.00 0.11 0.13
2  0.00 0.00 0.23 0.18 0.00 0.00 0.00 0.91 0.00 0.00 0.23 0.00 0.00 0.19 0.00
3  0.00 0.50 0.07 0.33 0.21 0.54 0.00 0.14 0.00 0.00 0.00 0.39 0.36 0.00 0.00
4  0.50 0.48 0.00 0.00 0.00 0.00 0.37 0.00 0.00 0.00 0.00 0.00 0.12 0.00 0.61
5  0.00 0.00 0.00 0.00 0.00 0.00 0.37 0.91 0.00 0.20 0.00 0.00 0.00 0.00 0.00
6  0.00 0.00 0.00 0.08 0.00 0.00 0.00 0.00 0.22 0.10 0.00 0.38 0.53 0.35 0.62
7  0.00 0.41 0.00 0.27 0.76 0.29 0.21 0.00 0.00 0.23 0.00 0.00 0.00 0.10 0.00
8  0.00 0.00 0.50 0.00 0.00 0.43 0.46 0.00 0.55 0.06 0.00 0.00 0.00 0.19 0.11
9  0.00 0.00 0.00 0.20 0.00 0.00 0.52 0.00 0.07 0.56 0.06 0.11 0.21 0.00 0.56
10 0.00 0.53 0.00 0.24 0.00 0.13 0.00 0.30 0.00 0.00 0.30 0.07 0.57 0.38 0.00
11 0.00 0.00 0.22 0.40 0.00 0.43 0.31 0.00 0.36 0.00 0.33 0.31 0.43 0.00 0.00
12 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0

In [26]:
%%R

docs

   X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15
1   1  0  4  0  0  2  0  0  2   0   0   6   0   1   1
2   0  0  1  1  0  0  0  4  0   0   1   0   0   1   0
3   0  6  1  6  2  9  0  2  0   0   0   6   6   0   0
4   5  6  0  0  0  0  6  0  0   0   0   0   2   0   9
5   0  0  0  0  0  0  4  9  0   2   0   0   0   0   0
6   0  0  0  1  0  0  0  0  2   1   0   4   6   4   6
7   0  6  0  6  9  6  4  0  0   4   0   0   0   2   0
8   0  0  9  0  0  9  9  0  9   1   0   0   0   4   2
9   0  0  0  4  0  0  9  0  1   9   1   2   4   0   9
10  0  6  0  4  0  2  0  4  0   0   4   1   9   6   0
11  0  0  4  9  0  9  6  0  6   0   6   6   9   0   0
12  0  0  0  0  0  0  0  0  0   0   0   0   1   0   6
13  0  0  0  4  0  6  2  6  0   0   2   6   2   1   0
14  0  0  1  1  0  2  0  4  0   4   2   0   0   2   0


## Cleaning text data

이전의 섹션에서 우리는 bag-of-words 모델, term frequencies, 그리고 tf-idfs를 배웠다. 하지만 첫번째 중요한 과정은 텍스트 데이터를 cleaning 하는 것이다(unwanted character때문). 왜 이게 중요한지 예시로 이해해보자.

In [6]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

여기서 볼 수 있듯이, HTML 마크업과 구두점 등이 들어가 있다. 구두점 등은 정보를 이용하는데 유용할 수 있지만, 여기서는 이모티콘을 제외한 모든 구두점과 HTML 마크업 등은 제거할 것이다. 이 일을 하려면, 파이썬의 **regular expression(regex)** library를 사용해야 한다.

In [50]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
                    ' '.join(emoticons).replace('-', '') #? 책에는 '.join으로 나와있음
    return text

첫번째 regex <[^>]*> 를 통해서, 우리는 HTML 마크업을 제거하였다. 많은 프로그래머들이 보통 HTML을 parsing하는데 regex를 쓰는 것을 좋아하지 않지만, 이 regex는 데이터셋을 cleaning하는데는 충분하다. HTML 마크업을 제거한 후, 이모티콘을 찾기 위해 `emoticons`라는 다른 regex를 이용했다. 다음은 `[\W]+`를 이용해 모든 non-word character를 지워 준 후, lowercase로 변환해 `emoticons`에 더해 주었다. 추가적으로, (-)캐릭터 또한 이모티콘에서 지워 주었다. 

참고: https://developers.google.com/edu/python/regular-expressions, https://docs.python.org/3.4/library/re.html

이모티콘 캐릭터들을 문서 문자열 뒤에 붙이는 것은 좋은 접근이 아니라고 보일수도 있지만, bag-of-words 모델에서는 문자의 순서가 중요하지 않기 때문에 이렇게 하더라도 큰 문제가 없다. 다음은 위 코드의 실행 샘플이다.

In [8]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [9]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

마지막으로, 우리는 `cleaned` 텍스트 데이터를 사용할 것이기 때문에 `preprocessor`함수를 우리의 데이터에 적용시켜준다.

In [51]:
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

*tokenize*를 위해 할수있는 한가지 방금은 스페이스를 기준으로 나누는 것이다.

In [11]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

다른 방법은 **word stemming**이라는 방법이다. 이는 단어를 그것의 root form으로 바꾸어 같은 root(stem)를 가진 단어끼리 mapping할 수있게 해준다. 우리가 여기서 사용하는 알고리즘은 **porter stemmer**이다. 이는  `pip install nltk`로 다운 가능하다.

NLTK관련 추가 정보: http://www.nltk.org/book/ 에서 확인 가능하다. porter stemmer은 꽤나 오래된 알고리즘이고, 다른 stemming 알고리즘 또한 존재한다. **Snowball stemmer**나 **Lancaster stemmer**가 그것이다. 이것들 또한 NLTK 패키지를 통해 이용 가능하다. (http://www.nltk.org/api/nltk.stem.html)

In [12]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

다른 유용한 주제인 **stop-word removal**에 대해 설명이다. Stop-words란 간단히 말해 모든 종류의 문서에 매우 자주 등장해 유용한 정보를 거의 담고 있지 않은 단어들이다. 예시로, is, and, has 등이 있다. 이러한 stop-words를 제거하는 것은 우리가 tf-idfs가 아닌 raw or normalized term frequencies를 가지고 작업할때 매우 유용하다. 

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WooJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## Training a logistic regression model for document classification

먼저, 훈련 데이터와 테스트 데이터셋을 25,000개씩으로 나눠준다.

In [15]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

`GridSearchCV`를 이용하여 최적의 파라메터 값을 찾아준다. (5-fold stratified cross-validation 이용)

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

param_grid = [{'vect__ngram_range': [(1,1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer, tokenizer_porter],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]},
                {'vect__ngram_range': [(1,1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer, tokenizer_porter],
                'vect__use_idf':[False],
                'vect__norm':[None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]}
]

lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                            scoring='accuracy',
                            cv=5, verbose=1,
                            n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

`GridSearchCV` 오브젝트를 사용할때 여기에서는 컴퓨팅 시간 때문에 파라메터의 경우의 수를 줄여 주었다. 또한 `CountVectorizer`와 `TfidfTransformer`를 `TfidVectorizer`로 바꿔 주었고, 이는 두가지 기능을 모두 수행한다. 여기서는 이전(`use_idf=True, smooth_idf=True, and norm='l2'`)과 달리 `use_idf=False, smooth_idf=False, and norm=None` 을 해 주었는데, 이는 raw term frequencies를 기반으로 모델을 훈련시키기 위함이다. 

In [53]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x117761d08>} 


<img src="ch8_9.png" alt="Drawing" style="width: 700px;"/>

Grid search에서 찾아낸 파라메터를 가지고 5-fold cross validation accuracy scores를 내어 보면,

In [ ]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

CV accuracy: 0.897,
Test accuracy: 0.899

Naive Bayes classifiers 정보: https://arxiv.org/pdf/1410.5329v3.pdf

## Working with bigger data - online algorithms and out-of-core learning

50,000개의 영화 리뷰 데이터로 feature vectors를 만들어 grid search를 하는건 꽤나 오랜 시간이 걸린다. 현실에서는 우리 컴퓨터의 메모리보다 큰 데이터를 가지고 작업을 하는 상황이 많다. 모든 사람이 슈퍼컴퓨터를 가지고 있지는 않기 때문에, out-or-core learning이라는 테크닉을 배워 볼 것이다. 이는 그러한 큰 데이터를 다룰 수 있게 해준다.

챕터2에서, 우리는 **stochastic gradient decent**라는 개념을 다뤘다. 이는 한번에 하나의 샘플을 이용해 모델의 weight를 업데이트 해주었다. 이 섹션에서는 `SGDClassifier`의 `partial_fit`라는 함수를 이용해 우리의 로컬 드라이브에서 바로 문서를 스트림해, small minibatches로 로지스틱 회귀를 훈련시켜볼 것이다.

먼저, `tokenizer`함수를 이용해 우리의 `movie_data.csv`파일의 unprocessed text data를 cleaning 해준다.

In [68]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
                    ' '.join(emoticons).replace('-', '') 
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

다음으로 `stream_docs`라는 generator function을 정의한다. 이는 한번에 한 문서를 리턴한다.

In [69]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [97]:
next(stream_docs(path='/Users/WooJin/Documents/ML_study/PML2/PML_Chp8/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

다음으로는 `get_minibatch`를 통해 문서의 지정된 숫자를 리턴한다.

In [77]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [102]:
next(doc_stream)

('"Every new fall line-up show deserves, at least, my ""3 strikes and you\'re out"" policy. I give a comedy 3 chances to make me laugh, that is, 3 complete episodes. After Episode 1, I actually said to the TV,""Cancelled tomorrow"". It was that bad. I have now watched the first 4 episodes of ""Cavemen"" and have yet to manage even a smirk. Not a titter, a guffaw, a chortle, as a matter of fact, no facial movement at all. I will continue to punish myself by watching every future episode because I am convinced that I am clearly missing something in this show. I\'m simply not ""getting"" it, but I believe that a comedy on a major TV network in prime-time, just HAS to be funny; but there are no laughs from me YET. There\'s just no way that ABC would put on the least funniest comedy of all time at 8:00 p.m. I KNOW there has got to be an inside joke that just isn\'t jiving with my brain. I\'ve read each of the previous comments, I ""get"" the social aspect of it, but, WHERE ARE THE JOKES ???

불행하게도, 우리는 out-of-core learning에서는 `CountVectorizer`를 사용할 수 없다. 왜냐면 이는 메모리에 모든 데이터를 담아두기 때문이다. 마찬가지로, `TfidVectorizer`또한 그렇다. 하지만 scikit-learn의 `HashingVectorizer`는 데이터와 무관하게 Hashing trick을 이용하기 때문에 이를 이용해 Vectorize해준다.

In [105]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                            n_features=2**21,
                            preprocessor=None,
                            tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)

doc_stream = stream_docs(path='/Users/WooJin/Documents/ML_study/PML2/PML_Chp8/movie_data.csv')

다음 코드를 이용해, 우리는 `tokenizer`수를 `HashingVectorizer`에 이용하였고, 로지스틱 회귀의 `loss` 파라메터를 `log`로 세팅했다. `HashingVectorizer`의 feature수가 많기 떄문에 우리는 hash collision을 줄일 수 있지만, 동시에 우리 모델의 coefficient 수를 늘리게 된다. --?

In [106]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:24


위 코드를 통해 45개의 minibatches(각각 1,000개의 문서)로 훈련시켰다. 남은 5,000개의 문서로 우리 모델의 성능을 평가하면,

In [107]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.867


여기서 볼 수 있듯, minibatch를 이용한 훈련 결과는 우리가 이전에 수행했던 grid search로 최적화된 모델보다는 더 적은 예측력을 가진다. 하지만, out-of-core learning은 굉장히 효율적이며 더 적은 시간이 걸린다. 마지막으로, 5,000개 문서를 이용해 우리의 모델을 업데이트 시켜준다.

In [108]:
clf = clf.partial_fit(X_test, y_test)

   - bag-of-words모델이 문서의 분류에는 많이 쓰이지만, 문장 구조나 문법은 신경쓰지 못한다. **Latent Dirichlet allocation**은 이를 보완하는 모델이다. 혹은 요즈음엔 **word2vec**을 이용하며, 이는 https://code.google.com/p/word2vec/ 를 참고할 수 있다.